# Problem Statement

- The primary motivation for us in coming up with this project was to apply a combination of the image manipulation/restoration techniques taught in this class (using skimage, etc.) with teachings from our other courses taken as a part of the MSIM program that had to do with Data Science.
- Having said so, we would like to acknowledge that we are all new to both the fields of image processing and deep learning.
- Now that that's established, we would like to explain the problem statement:
    - Taking old age grayscale/sepia images that have been creased/faded/etc. over time and applying image restoration techniques like denoising (to get rid of extra-graininess in images) & inpainting (to get rid of creases and fading caused over time).
    - Taking these restored images and colorizing them (to sort of simulate how the true colors of clothes, objects, landscape, etc. might have been in those times at the time of capture) using a deep learning model trained on a large set of images available on the internet.

# Dataset description

#### MIRFLICKR25k dataset : 
- 25,000 Flickr images under the Creative Commons license
- Each image is 224 by 224 pixels in size
- The images are converted into LAB space, then split into 2 parts - one with the L channel and the other with the AB channels

# Load all required libraries

#### Summary - 

- Numpy for stacking images in an array
- OpenCV for image restoration
- Matplotlib for general plotting tasks
- Skimage for image restoration
- Pandas for general python data manipulations
- Random for selecting random images from dataset for testing colorization feature
- PIL for Image manipulation
- OS for file manipulation, etc.
- TensorFlow for foundational support with creation of deep learning model
- Keras for high-level experimentation with deep learning model

In [ ]:
#import libraries

import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import skimage 
from skimage import io
import pandas as pd
import random
from PIL import Image
import skimage 
from skimage.transform import resize
from skimage.color import rgb2lab
import os
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Reshape, Dropout, LeakyReLU, BatchNormalization, Input, Concatenate, Activation, concatenate
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

# Define structure for a convolutional encoder-decoder deep learning model that colorizes grayscale images

The main inspiration for this approach was this paper published in 2017 - 
https://arxiv.org/abs/1712.03400

#### The below code block does following actions -

- Randomly initialize the weights for our neural network (to satisfy the expectation of SGD i.e. Stochastic Gradient Descent)
- Initialize an input layer that takes in images of the size 224 by 224 pixels
- Use the pretrained (imagenet data) neural network MobileNetV2 for high-level feature extraction
- Dropouts are implemented to prevent overfitting (this is done by dropping neurons at certain stages to train different neural networks each time)
- The output layer is configured for A and B channels (intended to be merged with the L channel) of our input images

In [120]:
# build a network for training on our dataset, use the pretrained MobileNet for deep layers

# prepare the kernel initializer values
weight_init = RandomNormal(stddev=0.02)

# prepare the Input layer
net_input = Input((224,224,3))

# download mobile net, and use it as the base.
mobile_net_base = MobileNetV2(include_top=False, input_shape=(224,224,3), weights='imagenet')
mobilenet = mobile_net_base(net_input)

# encoder block #

# 224x224
conv1 = Conv2D(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer=weight_init)(net_input)
conv1 = LeakyReLU(alpha=0.2)(conv1)

# 112x112
conv2 = Conv2D(128, (3, 3), strides=(1, 1), padding='same', kernel_initializer=weight_init)(conv1)
conv2 = LeakyReLU(alpha=0.2)(conv2)

# 112x112
conv3 = Conv2D(128, (3, 3), strides=(2, 2), padding='same', kernel_initializer=weight_init)(conv2)
conv3 =  Activation('relu')(conv3)

# 56x56
conv4 = Conv2D(256, (3, 3), strides=(2, 2), padding='same', kernel_initializer=weight_init)(conv3)
conv4 = Activation('relu')(conv4)

# 28x28
conv4_ = Conv2D(256, (3, 3), strides=(1, 1), padding='same', kernel_initializer=weight_init)(conv4)
conv4_ = Activation('relu')(conv4_)

# 28x28
conv5 = Conv2D(512, (3, 3), strides=(2, 2), padding='same', kernel_initializer=weight_init)(conv4_)
conv5 = Activation('relu')(conv5)

# 14x14
conv5_ = Conv2D(256, (3, 3), strides=(2, 2), padding='same', kernel_initializer=weight_init)(conv5)
conv5_ = Activation('relu')(conv5_)

#7x7
# fusion layer - connect MobileNet with our encoder
conc = concatenate([mobilenet, conv5_])
fusion = Conv2D(512, (1, 1), padding='same', kernel_initializer=weight_init)(conc)
fusion = Activation('relu')(fusion)

# skip fusion layer
skip_fusion = concatenate([fusion, conv5_])

# decoder block #

# 7x7
decoder = Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same', kernel_initializer=weight_init)(skip_fusion)
decoder = Activation('relu')(decoder)
decoder = Dropout(0.25)(decoder)

# skip layer from conv5 (with added dropout)
skip_4_drop = Dropout(0.25)(conv5)
skip_4 = concatenate([decoder, skip_4_drop])

# 14x14
decoder = Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same', kernel_initializer=weight_init)(skip_4)
decoder = Activation('relu')(decoder)
decoder = Dropout(0.25)(decoder)

# skip layer from conv4_ (with added dropout)
skip_3_drop = Dropout(0.25)(conv4_)
skip_3 = concatenate([decoder, skip_3_drop])

# 28x28
decoder = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', kernel_initializer=weight_init)(skip_3)
decoder = Activation('relu')(decoder)
decoder = Dropout(0.25)(decoder)

# 56x56
decoder = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer=weight_init)(decoder)
decoder = Activation('relu')(decoder)
decoder = Dropout(0.25)(decoder)

# 112x112
decoder = Conv2DTranspose(64, (3, 3), strides=(1, 1), padding='same', kernel_initializer=weight_init)(decoder)
decoder = Activation('relu')(decoder)

# 112x112
decoder = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', kernel_initializer=weight_init)(decoder)
decoder = Activation('relu')(decoder)

# 224x224
# output layer, with 2 channels (a and b)
output_layer = Conv2D(2, (1, 1), activation='tanh')(decoder)

# Configure model to be optimized using Adam optimizer, specified learning rate and mean-squared error loss

- This step loads the weights derived upon training our deep learning model for 100 epochs on the entire MIRFLICKR25k dataset (in batches of 3000 each)

In [121]:
# configure model
model = Model(net_input, output_layer)
model.compile(Adam(lr=0.0002), loss='mse', metrics=['accuracy'])

# load weights
model.load_weights('trained_on_all_is_for_100_es.h5')

# Define all functions for getting predictions and constructing complete colorized RGB image based on LAB inputs

In [10]:
def get_pred(model, image_l):
    """
    Summary -
    This function generates the predicted A and B channels of the colorized image

    :param model: Trained model
    :type model: tensorflow.python.keras.engine.training.Model
    :param image_l: The L channel of the input grayscale image
    :type image_l: numpy.ndarray
    :return: Predicted A and B channels of the input grayscale image
    :rtype: numpy.ndarray
    
    """
    # repeat the L value to match input shape
    image_l_R = np.repeat(image_l[..., np.newaxis], 3, -1)
    image_l_R = image_l_R.reshape((1, 224, 224, 3))
    # normalize the input
    image_l_R = (image_l_R.astype('float32') - 127.5) / 127.5
    # make prediction
    prediction = model.predict(image_l_R)
    # normalize the output
    pred = (prediction[0].astype('float32') * 127.5) + 127.5
    return pred

def get_LAB(image_l, image_ab):
    """
    Summary - 
    This function generates the compiled RGB equivalent of the inputted L plus the predicted A and B channels

    :param image_l: Grayscale image (1 layer, L)
    :type image_l: numpy.ndarray
    :param image_ab: Predicted image (2 layers, A and B)
    :type image_ab: numpy.ndarray
    :return: Compiled image (3 layers, RGB)
    :rtype: numpy.ndarray
    
    """
    image_l = image_l.reshape((224, 224, 1))
    image_lab = np.concatenate((image_l, image_ab), axis=2)
    image_lab = image_lab.astype("uint8")
    image_rgb = cv.cvtColor(image_lab, cv.COLOR_LAB2RGB)
    image_rgb = Image.fromarray(image_rgb)
    return image_rgb

def create_sample(model, image_gray):
    """
    Summary - 
    This function creates the output we need from colorization

    :param model: Trained model
    :type model: tensorflow.python.keras.engine.training.Model
    :param image_gray: Grayscale image (1 layer, L)
    :type image_gray: numpy.ndarray
    :return: Result image
    :rtype: numpy.ndarray
    
    """
    # get the model's prediction
    pred = get_pred(model, image_gray)
    # combine input and output to LAB image
    image = get_LAB(image_gray, pred)
    # create new combined image, save it
    new_image = Image.new('RGB', (224, 224))
    new_image.paste(image, (0, 0))
    return new_image

# Define functions for image restoration, etc.

In [175]:
def show_image(Selection,l,i):
    """
    Summary -
    Selects the image based on user choice and sends it to the deep learning model for processing
    
    """
    index = l.index(Selection)
    img = i[index]
    mod_img = resize(rgb2lab(img.copy())[:,:,0], (224,224), anti_aliasing=True)
    sample = create_sample(model, mod_img)
    sample.save('output/output.jpg')
    
    fig, axes = plt.subplots(ncols = 2, figsize = (15,5))
    axes[0].imshow(img)
    axes[0].axis('off')
    axes[0].set_title(Selection)

    axes[1].imshow(sample)
    axes[1].axis('off')
    axes[1].set_title('Auto-Colored')

In [95]:
def img_restoration(img):
    """
    Summary -
    This function is created to carry out image restoration process in steps
    
    """
    #perform denoising of the image
    denoised = cv.fastNlMeansDenoisingColored(img,None,7,10,6,21)
    #canny edge detection
    edges = cv.Canny(denoised,200,250)
    #filter for image processing
    kernel = np.ones((5,5),np.uint8)
    #image dilation
    dilation = cv.morphologyEx(edges, cv.MORPH_DILATE, kernel)
    closing = cv.morphologyEx(dilation, cv.MORPH_CLOSE, kernel)
    erode = cv.morphologyEx(closing,cv.MORPH_ERODE, kernel)
    #fill in missing gaps 
    inpainted = cv.inpaint(denoised,erode,5,cv.INPAINT_TELEA)
    #overlay denoised image over smoothed out image
    alpha = 0.5
    overlaid = inpainted.copy()
    cv.addWeighted(denoised, alpha, overlaid, 1 - alpha,0, overlaid)
    
    #convert image to gray
    img2gray = cv.cvtColor(denoised,cv.COLOR_BGR2GRAY)
    #create mask and inverse mask based on threshold
    ret, mask = cv.threshold(img2gray, 100, 255, cv.THRESH_BINARY_INV)
    #combine via bit addition denoised image human and smoothed out background of inpainted image
    bg1 = cv.bitwise_and(denoised,denoised,mask = mask)
    mask_inv = cv.bitwise_not(mask)
    bg2 = cv.bitwise_and(overlaid,overlaid,mask = mask_inv)
    combined = cv.add(bg1,bg2)
    
    #store the various processed images
    images = [img,denoised,inpainted,overlaid,combined]
    labels = ['Original','Choice 1','Choice 2', 'Choice 3', 'Choice 4']
    return images, labels

In [96]:
def display_images(images,labels):
    """
    Summary - 
    This function displays the various processed images and labels associated with them
    
    """
    fig, axes = plt.subplots(ncols = 5, figsize = (15,5))
    axes[0].imshow(images[0])
    axes[0].axis('off')
    axes[0].set_title(labels[0])

    axes[1].imshow(images[1])
    axes[1].axis('off')
    axes[1].set_title(labels[1])
    
    axes[2].imshow(images[2])
    axes[2].axis('off')
    axes[2].set_title(labels[2])
    
    axes[3].imshow(images[3])
    axes[3].axis('off')
    axes[3].set_title(labels[3])
    
    axes[4].imshow(images[4])
    axes[4].axis('off')
    axes[4].set_title(labels[4])

In [42]:
#!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [176]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interact_manual, fixed, Output

#create button for widget
button = widgets.Button(description="Begin Restoration")
save = widgets.Button(description="Save Result")
#create text box for widget
filename = widgets.Text(value='<filename>.jpg',placeholder='Type something',disabled=False)
#create output area for widget
output = widgets.Output()

#layout setting piece for code
vertical = widgets.VBox([widgets.Label(value="Enter the name of the picture you want to restore:"),widgets.HBox([filename, button])])
display(vertical)
display(output)


def on_button_clicked(b):
    """
    Summary -
    This function handles the button clicked event
    
    """
    with output:
        clear_output()
        #read the image
        img = io.imread("images/"+filename.value)
        #gets 4 different kinds of processed images
        images, labels = img_restoration(img)
        #displays the various options
        display_images(images,labels)
        display(widgets.Label(value="Select the picture you want to color:"))
        #code that calls the model based on user selected image
        interact(show_image, Selection = labels, l=fixed(labels), i=fixed(images), description = 'Choose image to color')
        display(widgets.Label(value="Output saved to the output folder"))

button.on_click(on_button_clicked)

Output()